In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, classification_report
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
print("데이터 로드 중...")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

print(f"학습 데이터 크기: {train_df.shape}")
print(f"테스트 데이터 크기: {test_df.shape}")
print(f"제출 샘플 크기: {sample_submission.shape}")

# 기본적인 데이터 정보 확인
print("\n학습 데이터 정보:")
print(train_df.info())

print("\n결측치 확인:")
print(train_df.isnull().sum())

# 타겟 변수 분포 확인
print("\n타겟 변수(Cancer) 분포:")
print(train_df['Cancer'].value_counts())
print(f"암 양성 비율: {train_df['Cancer'].mean() * 100:.2f}%")

# 1. EDA & 전처리
print("\n데이터 전처리 진행 중...")

# ID 열 drop
train_df_processed = train_df.drop('ID', axis=1)
test_df_processed = test_df.drop('ID', axis=1)

# 범주형(문자열) 변수와 수치형 변수 분리
categorical_cols = train_df_processed.select_dtypes(include=['object']).columns
numerical_cols = train_df_processed.select_dtypes(exclude=['object']).columns.drop('Cancer')

print(f"\n범주형 변수: {list(categorical_cols)}")
print(f"수치형 변수: {list(numerical_cols)}")

# 2. 개선된 EDA 및 특성 엔지니어링
print("\n개선된 특성 엔지니어링 진행 중...")

# 2.1 아이오딘 결핍과 국가 상호작용 특성 생성
train_df_processed['Iodine_Country_Risk'] = ((train_df_processed['Iodine_Deficiency'] == 'Deficient') & 
                                            (train_df_processed['Country'] == 'IND')).astype(int)
test_df_processed['Iodine_Country_Risk'] = ((test_df_processed['Iodine_Deficiency'] == 'Deficient') & 
                                           (test_df_processed['Country'] == 'IND')).astype(int)

# 2.2 수치형 변수의 이상치 클리핑
for col in numerical_cols:
    q1 = train_df_processed[col].quantile(0.01)
    q3 = train_df_processed[col].quantile(0.99)
    train_df_processed[col] = train_df_processed[col].clip(q1, q3)
    test_df_processed[col] = test_df_processed[col].clip(q1, q3)

# 데이터 준비
X = train_df_processed.drop('Cancer', axis=1)
y = train_df_processed['Cancer']

# 학습 데이터와 검증 데이터 분리 (20% 검증 데이터)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 수치형 변수 정규화
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
test_df_processed[numerical_cols] = scaler.transform(test_df_processed[numerical_cols])

# SMOTE 적용을 위한 범주형 변수 임시 인코딩 (LabelEncoder)
# SMOTE는 범주형 변수를 처리할 수 없으므로 SMOTE 적용 전에 인코딩
print("\n범주형 변수를 숫자로 임시 변환하여 SMOTE 적용...")
X_train_encoded = X_train.copy()
X_val_encoded = X_val.copy()

# 범주형 변수를 인코딩하기 위한 LabelEncoder 사전
label_encoders = {}

# 범주형 변수 인코딩
for col in categorical_cols:
    le = LabelEncoder()
    # 학습 데이터와 검증 데이터의 범주를 모두 포함하여 fit
    le.fit(pd.concat([X_train[col], X_val[col]]))
    X_train_encoded[col] = le.transform(X_train[col])
    X_val_encoded[col] = le.transform(X_val[col])
    label_encoders[col] = le

# 3. 모델 구축
print("\n모델 구축 중...")

# SMOTE를 사용하여 클래스 불균형 처리 (인코딩된 데이터로)
print("SMOTE 적용하여 클래스 불균형 해소...")
smote = SMOTE(random_state=42)
X_train_smote_encoded, y_train_smote = smote.fit_resample(X_train_encoded, y_train)

# SMOTE 후 결과 확인
print(f"SMOTE 적용 전 학습 데이터 크기: {X_train.shape}, 양성 비율: {y_train.mean() * 100:.2f}%")
print(f"SMOTE 적용 후 학습 데이터 크기: {X_train_smote_encoded.shape}, 양성 비율: {y_train_smote.mean() * 100:.2f}%")

# SMOTE 후 다시 원래 형태로 복원하기 위해 데이터프레임 생성
X_train_smote = pd.DataFrame(X_train_smote_encoded, columns=X_train.columns)

# 범주형 변수를 다시 원래 형태(문자열)로 변환 - CatBoost에서 사용하기 위함
for col in categorical_cols:
    X_train_smote[col] = label_encoders[col].inverse_transform(X_train_smote_encoded[col].astype(int))

print(f"SMOTE 적용 전 학습 데이터 크기: {X_train.shape}, 양성 비율: {y_train.mean() * 100:.2f}%")
print(f"SMOTE 적용 후 학습 데이터 크기: {X_train_smote.shape}, 양성 비율: {y_train_smote.mean() * 100:.2f}%")

# CatBoost 모델 학습
print("\nCatBoost 모델 학습 중...")

# 범주형 변수 인덱스 생성
cat_features = [X_train_smote.columns.get_loc(col) for col in categorical_cols]

# CatBoost 모델 설정
catboost_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    loss_function='Logloss',
    eval_metric='F1',
    random_seed=42,
    verbose=100,
    task_type='CPU'
    # SMOTE로 클래스 불균형이 해소되었으므로 class_weights는 제거
)

# Pool 객체 생성 (CatBoost 권장 방식)
train_pool = Pool(data=X_train_smote, 
                  label=y_train_smote,
                  cat_features=cat_features)

val_pool = Pool(data=X_val,
                label=y_val,
                cat_features=cat_features)

# 모델 학습
catboost_model.fit(
    train_pool,
    eval_set=val_pool,
    early_stopping_rounds=50,
    use_best_model=True
)

# 검증 데이터로 성능 평가
y_val_pred = catboost_model.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred)

print("\n검증 데이터에 대한 성능 평가:")
print(classification_report(y_val, y_val_pred))
print(f"Validation F1-Score: {val_f1:.4f}")

# 특성 중요도 시각화
plt.figure(figsize=(14, 10))
feature_importance = catboost_model.get_feature_importance()
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False).head(20)

plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title('CatBoost Feature Importance (Top 20)')
plt.xlabel('Importance')
plt.gca().invert_yaxis()  # 중요도가 높은 특성을 위에 표시
plt.tight_layout()
plt.savefig('catboost_feature_importance.png')
plt.close()

# 테스트 데이터에 예측
print("\n테스트 데이터에 예측 중...")
test_pool = Pool(data=test_df_processed, cat_features=cat_features)
test_predictions = catboost_model.predict(test_pool)

# sample_submission 파일에 예측 결과 저장
sample_submission['Cancer'] = test_predictions.astype(int)
sample_submission.to_csv('catboost_submission.csv', index=False)

print("\n예측 완료! 'catboost_submission.csv' 파일에 결과가 저장되었습니다.")

# 예측 결과 확인
print("예측 결과 분포:")
print(sample_submission['Cancer'].value_counts())
print(f"암 양성 예측 비율: {sample_submission['Cancer'].mean() * 100:.2f}%")

데이터 로드 중...
학습 데이터 크기: (87159, 16)
테스트 데이터 크기: (46204, 15)
제출 샘플 크기: (46204, 2)

학습 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87159 entries, 0 to 87158
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 87159 non-null  object 
 1   Age                87159 non-null  int64  
 2   Gender             87159 non-null  object 
 3   Country            87159 non-null  object 
 4   Race               87159 non-null  object 
 5   Family_Background  87159 non-null  object 
 6   Radiation_History  87159 non-null  object 
 7   Iodine_Deficiency  87159 non-null  object 
 8   Smoke              87159 non-null  object 
 9   Weight_Risk        87159 non-null  object 
 10  Diabetes           87159 non-null  object 
 11  Nodule_Size        87159 non-null  float64
 12  TSH_Result         87159 non-null  float64
 13  T4_Result          87159 non-null  float64
 14  T3_Result          87159 n